# Class 17 - Solution Code

SQL Databases

In [1]:
import pandas as pd
from pandas.io import sql

Pandas can be used to connect to most relational databases. In this demonstration, we will create and connect to a SQLite database. SQLite creates portable SQL databases saved in a single file. These databases are stored in a very efficient manner and allow fast querying, making them ideal for small databases or databases that need to be moved across machines.

If you are looking to start using a database without the setup of `mysql` or `postgres`, SQLite is a good start.

# Part 1: SQLite DB for Rossman Store Data

[SQLite](http://www.sqlite.org)

Let's return to the Rossmann sales data and load that into the database.

Data is moved to the database through the `to_sql` command, similar to the `to_csv` command.

Pandas `to_sql` takes as arugments:
    - `name`, the table name to create
    - `con`, a connection to a database
    - `index`, whether to input the index column
    - `schema`, if we want to write a custom schema for the new table
    - `if_exists`, what to do if the table already exists. We can overwrite it, add to it, or fail
    
SQLite can execute the following SQL commands: [https://www.sqlite.org/lang.html](https://www.sqlite.org/lang.html)

In [2]:
import sqlite3

In [3]:
# Create a new SQLite DB and establish connection
conn = sqlite3.connect('dat-test.db')

### Load the Rossmann Store sales data in `rossmann.csv` and create a table into the database

In [4]:
# Load data from CSV
rossmann_sales = pd.read_csv('../../../lesson-15/assets/dataset/rossmann.csv', low_memory=False)
rossmann_sales.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [5]:
# Write data to DB
rossmann_sales.to_sql('rossmann_sales',
            con=conn,
            if_exists='replace',
            index=False)

### Check table schema for `rossmann_sales`

In [6]:
sql.read_sql(
"""
PRAGMA table_info('rossmann_sales');
""", con=conn)

,cid,name,type,notnull,dflt_value,pk
0,0,Store,INTEGER,0,None,0
1,1,DayOfWeek,INTEGER,0,None,0
2,2,Date,TEXT,0,None,0
3,3,Sales,INTEGER,0,None,0
4,4,Customers,INTEGER,0,None,0
5,5,Open,INTEGER,0,None,0
6,6,Promo,INTEGER,0,None,0
7,7,StateHoliday,TEXT,0,None,0
8,8,SchoolHoliday,INTEGER,0,None,0


### Query all columns in `rossmann_sales`

In [7]:
# Load data from DB
sql.read_sql('select * from rossmann_sales limit 5', con=conn)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


### CHECK: Load the Rossmann Store metadata in `rossmann-stores.csv` and create a table into the database

In [8]:
# Load data from CSV
rossmann_stores = pd.read_csv('../../assets/dataset/rossmann-stores.csv')

# Write data to DB
rossmann_stores.to_sql('rossmann_stores', if_exists='replace', index=False, con=conn)

### CHECK: Check table schema for `rossmann_stores`

In [9]:
sql.read_sql(
"""
PRAGMA table_info('rossmann_stores');
""", con=conn)

,cid,name,type,notnull,dflt_value,pk
0,0,Store,INTEGER,0,None,0
1,1,StoreType,TEXT,0,None,0
2,2,Assortment,TEXT,0,None,0
3,3,CompetitionDistance,REAL,0,None,0
4,4,CompetitionOpenSinceMonth,REAL,0,None,0
5,5,CompetitionOpenSinceYear,REAL,0,None,0
6,6,Promo2,INTEGER,0,None,0
7,7,Promo2SinceWeek,REAL,0,None,0
8,8,Promo2SinceYear,REAL,0,None,0
9,9,PromoInterval,TEXT,0,None,0


### CHECK: Query all columns in `rossmann_stores`

In [10]:
# Load data from DB
sql.read_sql('select * from rossmann_stores limit 5', con=conn)

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,None
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,None
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,None


### DEMO: (SELECT) Write a query that returns the `Store`, `Date` and `Customers`

In [11]:
sql.read_sql(
"""
SELECT 
Store, Date, Customers 
FROM rossmann_sales
LIMIT 5;
""", con=conn)

,Store,Date,Customers
0,1,2015-07-31,555
1,2,2015-07-31,625
2,3,2015-07-31,821
3,4,2015-07-31,1498
4,5,2015-07-31,559


### DEMO: (WHERE) Have the students write a query that returns the `Store`, `Date` and `Customers` for when the stores were open and running a promotion

In [12]:
sql.read_sql(
"""
SELECT 
Store, Date, Customers 
FROM rossmann_sales
WHERE Open = 1 and Promo = 1;
""", con=conn).head()

,Store,Date,Customers
0,1,2015-07-31,555
1,2,2015-07-31,625
2,3,2015-07-31,821
3,4,2015-07-31,1498
4,5,2015-07-31,559


### DEMO: (GROUP BY) Have the students write a query that returns the total sales on promotion days.

In [13]:
sql.read_sql(
"""
SELECT
Promo, SUM(Sales)
FROM rossmann_sales
GROUP BY Promo
""", con=conn).head()

,Promo,SUM(Sales)
0,0,2771974337
1,1,3101206286


### DEMO: (JOIN) Select Store, Customers, from `rossmann_sales` and StoreType from `rossmann_stores`

In [14]:
# Select the Store, Customers, and StoreType
query = """
SELECT 
t1.Store, t1.Customers, t2.StoreType
FROM rossmann_sales as t1
JOIN rossmann_stores as t2
ON t1.Store = t2.Store
"""
sql.read_sql(query, con = conn).head()

,Store,Customers,StoreType
0,1,555,c
1,2,625,a
2,3,821,a
3,4,1498,c
4,5,559,a


# Part 2: [EXERCISE] SQLite DB for Walmart Store Data

### Exercises
1. Load the Walmart sales and store features data
1. Create a table for each of those datasets
1. Select the store, date and fuel price on days it was over 90 degrees
1. Select the store, date and weekly sales and temperature
1. What were average sales on holiday vs. non-holiday sales
1. What were average sales on holiday vs. non-holiday sales when the temperature was below 32 degrees

### 2.1 Load the Walmart sales and store features data

In [15]:
%time

# Load the Walmart sales and store features data
walmart_sales = pd.read_csv('../../assets/dataset/walmart-sales.csv')
walmart_sales.head()

walmart_features = pd.read_csv('../../assets/dataset/features.csv')
walmart_features.head()

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.01 µs


,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


### 2.2 Create a table for each of those datasets

In [16]:
%time

# Create a table for each of those datasets
walmart_sales.to_sql('walmart_sales', if_exists='replace', index=False, con = conn)
walmart_features.to_sql('walmart_features', if_exists='replace', index=False, con=conn)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs


In [17]:
%time

sql.read_sql(
"""
PRAGMA table_info('walmart_sales');
""", con=conn)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.05 µs


,cid,name,type,notnull,dflt_value,pk
0,0,Store,INTEGER,0,None,0
1,1,Dept,INTEGER,0,None,0
2,2,Date,TEXT,0,None,0
3,3,Weekly_Sales,REAL,0,None,0
4,4,IsHoliday,INTEGER,0,None,0


In [18]:
%time

sql.read_sql(
"""
PRAGMA table_info('walmart_features');
""", con=conn)

CPU times: user 2 µs, sys: 3 µs, total: 5 µs
Wall time: 7.87 µs


,cid,name,type,notnull,dflt_value,pk
0,0,Store,INTEGER,0,None,0
1,1,Date,TEXT,0,None,0
2,2,Temperature,REAL,0,None,0
3,3,Fuel_Price,REAL,0,None,0
4,4,MarkDown1,REAL,0,None,0
5,5,MarkDown2,REAL,0,None,0
6,6,MarkDown3,REAL,0,None,0
7,7,MarkDown4,REAL,0,None,0
8,8,MarkDown5,REAL,0,None,0
9,9,CPI,REAL,0,None,0


### 2.3 Select the store, date and fuel price on days it was over 90 degrees

HINT: Use WHERE

In [19]:
%time

# Select the store, date and fuel price on days it was over 90 degrees
query = """
SELECT 
Date, Store, Fuel_Price, Temperature
FROM walmart_features
WHERE Temperature > 90
"""
sql.read_sql(query, con = conn).head()

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.01 µs


,Date,Store,Fuel_Price,Temperature
0,2011-08-05,1,3.684,91.65
1,2011-08-12,1,3.638,90.76
2,2011-07-29,2,3.682,90.07
3,2011-08-05,2,3.684,93.34
4,2011-08-12,2,3.638,91.58


### 2.4 Select the store, date and weekly sales and temperature

HINT: Use JOIN

In [20]:
%time

# Select the store, date and weekly sales and temperature
query = """
SELECT 
s.Store, s.Date, s.Weekly_Sales, f.Temperature
FROM walmart_sales as s
JOIN walmart_features as f
ON s.Store = f.Store and s.Date = f.Date
"""
sql.read_sql(query, con = conn).head()

CPU times: user 4 µs, sys: 3 µs, total: 7 µs
Wall time: 7.87 µs


,Store,Date,Weekly_Sales,Temperature
0,1,2010-02-05,24924.50,42.31
1,1,2010-02-12,46039.49,38.51
2,1,2010-02-19,41595.55,39.93
3,1,2010-02-26,19403.54,46.63
4,1,2010-03-05,21827.90,46.50


### 2.5 What were average sales on holiday vs. non-holiday sales

HINT: Use GROUP BY

In [21]:
%time

# What were average sales on holiday vs. non-holiday sales
query = """
SELECT 
IsHoliday, AVG(Weekly_Sales)
FROM walmart_sales as s
GROUP BY IsHoliday
"""
sql.read_sql(query, con = conn)

CPU times: user 3 µs, sys: 2 µs, total: 5 µs
Wall time: 7.15 µs


,IsHoliday,AVG(Weekly_Sales)
0,0,15901.445069
1,1,17035.823187


### 2.6 What were average sales on holiday vs. non-holiday sales when the temperature was below 32 degrees

HINT: Use JOIN, WHERE, and GROUP BY

In [22]:
%time

# What were average sales on holiday vs. non-holiday sales when the temperature was below 32 degrees
query = """
SELECT 
s.IsHoliday, AVG(s.Weekly_Sales)
FROM walmart_sales as s
JOIN walmart_features as f
ON s.Store = f.Store and s.Date = f.Date
WHERE f.Temperature < 32
GROUP BY s.IsHoliday
"""
sql.read_sql(query, con = conn)

CPU times: user 5 µs, sys: 7 µs, total: 12 µs
Wall time: 12.2 µs


,IsHoliday,AVG(s.Weekly_Sales)
0,0,15275.770307
1,1,15111.471238
